# From Decision Trees to Random Forests

```
Authors: Alexandre Gramfort
         Thomas Moreau
```

## Bagging classifiers

We saw that by increasing the depth of the tree, we are going to get an over-fitted model. A way to bypass the choice of a specific depth it to combine several trees together.

Let's start by training several trees on slightly different data. The slightly different dataset could be generated by randomly sampling with replacement. In statistics, this called a boostrap sample. We will use the iris dataset to create such ensemble and ensure that we have some data for training and some left out data for testing.

In [1]:
import numpy as np

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

Before to train several decision trees, we will run a single tree. However, instead to train this tree on `X_train`, we want to train it on a bootstrap sample. You can use the `np.random.choice` function sample with replacement some index. You will need to create a sample_weight vector and pass it to the `fit` method of the `DecisionTreeClassifier`. We provide the `generate_sample_weight` function which will generate the `sample_weight` array.

In [2]:
def bootstrap_idx(X):
    indices = np.random.choice(
        np.arange(X.shape[0]), size=X.shape[0], replace=True
    )
    return indices

In [3]:
bootstrap_idx(X_train)

array([ 67,  21,  81,  15,  35,  49,  27,   0,  61,  14,  21,   7,  75,
        78,   5,  26, 104,  75,  18,  54,  18,  76,  49, 107,   4,  73,
       106,  34,  58,  78,  91,  77, 110,  95, 106,   1,  15,  83,  32,
       109,   1,  21,  24,  64,  31,  13,  11,  21,   7,  71,  25,  48,
        52,  62,  67,  86,  91,  90,  61,  56,  66,  28,   0,  78,  92,
        42,  14,  15,  80,  24,  67,  73,  90,  73,  40,  16,  30,   8,
        41,  88,   5,  92,  51,  26,  25,  39,  68,  57, 104,  40,  45,
        85,  17,  48,  85,  72, 106,  38, 101,  51,  40,  33,  56,  16,
        56,  72,  46,  38,  70,  50,   6,  77])

In [4]:
from collections import Counter
Counter(bootstrap_idx(X_train))

Counter({100: 1,
         3: 2,
         70: 2,
         7: 1,
         2: 2,
         56: 3,
         12: 2,
         96: 3,
         57: 3,
         25: 3,
         86: 1,
         67: 1,
         101: 1,
         54: 1,
         40: 1,
         75: 1,
         37: 3,
         50: 1,
         79: 2,
         82: 3,
         106: 1,
         26: 2,
         52: 1,
         8: 1,
         68: 3,
         69: 2,
         32: 3,
         104: 3,
         76: 1,
         71: 2,
         11: 1,
         89: 1,
         10: 2,
         109: 1,
         84: 1,
         44: 1,
         83: 1,
         22: 1,
         77: 1,
         17: 2,
         53: 1,
         66: 1,
         4: 1,
         91: 1,
         31: 1,
         29: 2,
         61: 1,
         35: 2,
         95: 1,
         14: 1,
         23: 1,
         16: 1,
         43: 3,
         72: 1,
         73: 1,
         39: 1,
         9: 1,
         108: 2,
         78: 1,
         58: 1,
         81: 2,
         85: 1,
        

In [5]:
def bootstrap_sample(X, y):
    indices = bootstrap_idx(X)
    return X[indices], y[indices]

In [6]:
X_train_bootstrap, y_train_bootstrap = bootstrap_sample(X_train, y_train)

In [7]:
print(f'Classes distribution in the original data: {Counter(y_train)}')
print(f'Classes distribution in the bootstrap: {Counter(y_train_bootstrap)}')

Classes distribution in the original data: Counter({0: 38, 1: 37, 2: 37})
Classes distribution in the bootstrap: Counter({0: 51, 1: 31, 2: 30})


<div class="alert alert-success">
    <b>EXERCISE: Create a bagging classifier</b>:<br>
    <br>
    A bagging classifier will train several decision tree classifiers, each of them on a different bootstrap sample.
     <ul>
      <li>
      Create several <code>DecisionTreeClassifier</code> and store them in a Python list;
      </li>
      <li>
      Loop over these trees and <code>fit</code> them by generating a bootstrap sample using <code>bootstrap_sample</code> function;
      </li>
      <li>
      To predict with this ensemble of trees on new data (testing set), you can provide the same set to each tree and call the <code>predict</code> method. Aggregate all predictions in a NumPy array;
      </li>
      <li>
      Once the predictions available, you need to provide a single prediction: you can retain the class which was the most predicted which is called a majority vote;
      </li>
      <li>
      Finally, check the accuracy of your model.
      </li>
    </ul>
</div>

In [8]:
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import mode
from sklearn.metrics import accuracy_score

ntrees = 5
max_depth = 3

tree_list = []
ypred_list = []
for i in range(ntrees):
    X_train_i, y_train_i = bootstrap_sample(X_train, y_train)
    treei = DecisionTreeClassifier(max_depth=max_depth)
    treei.fit(X_train_i, y_train_i)
    ypred_list.append(treei.predict(X_test))
    tree_list.append(treei)
    
ypred,_ = mode(np.array(ypred_list), axis=0, keepdims=False)
print(accuracy_score(y_test, ypred))

0.9473684210526315


<div class="alert alert-success">
    <b>EXERCISE: using scikit-learn</b>:
    <br>
    After implementing your own bagging classifier, use a <code>BaggingClassifier</code> from scikit-learn to fit the above data.
</div>

## Random Forests

A very famous classifier is the random forest classifier. It is similar to the bagging classifier. In addition of the bootstrap, the random forest will use a subset of features (selected randomly) to find the best split.

<div class="alert alert-success">
    <b>EXERCISE: Create a random forest classifier</b>:
    <br>
    Use your previous code which was generated several <code>DecisionTreeClassifier</code>. Check the list of the option of this classifier and modify one of the parameters such that only the $\sqrt{F}$ features are used for the splitting. $F$ represents the number of features in the dataset.
</div>

<div class="alert alert-success">
    <b>EXERCISE: using scikit-learn</b>:
    <br>
    After implementing your own random forest classifier, use a <code>RandomForestClassifier</code> from scikit-learn to fit the above data.
</div>

In [9]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=5)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.9210526315789473

In [10]:
from figures import plot_forest_interactive
plot_forest_interactive()

interactive(children=(IntSlider(value=0, description='max_depth', max=8), Output()), _dom_classes=('widget-int…